In [5]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from dataclasses import dataclass

In [6]:
def matComp(xmin,xmax,ymin,ymax,dens):
    re=np.linspace(xmax,xmin,int(xmax-xmin)*dens)
    im=np.linspace(ymax,ymin,int(ymax-ymin)*dens)
    return re[np.newaxis,:]+im[:,np.newaxis]*1j

In [7]:
def esEstable(c,N):
    z=0
    for _ in range(N):
        z=z**2+c
        if abs(z) > 2:
            return False
    return True

In [8]:
def getMandelbrot_chafa(c,N):
    mask=esEstable(c,N)
    return c[mask]

In [11]:
@dataclass
class ConjMandelbrot:
    max_iter:int
    def __cont__(self,c):
        return self.stability(c)==1
    def stability(self,c):
        return self.esc_count(c)/self.max_iter
    def esc_count(self,c):
        z=0
        for i in range(self.max_iter):
            z=z**2+c
            if abs(z)>2:
                return i
        return self.max_iter


In [12]:
mandel=ConjMandelbrot(max_iter=30)

In [13]:
mandel.esc_count(0.25)

30

In [14]:
mandel.stability(0.25)

1.0

In [16]:
mandel.__cont__(0.25)

True

In [19]:
mandel.esc_count(0.26)
#mandel.stability(0.26)
#mandel.__cont__(0.26)

29

In [29]:
mandel=ConjMandelbrot(max_iter=20)
w,h=512,512
scale=0.0075
im=Image.new(mode="L",size=(w,h))
for y in range(h):
    for x in range(w):
        c=scale*complex(x-w/2,h/2-y)
        instability = 1-mandel.stability(c)
        im.putpixel((x,y),int(instability * 255)  )


In [30]:
im.show()

In [44]:
from math import log
@dataclass
class ConjMandelbrot:
    max_iter:int
    radio_esc:float
    def __cont__(self,c):
        return self.stability(c)==1
    def stability(self,c,smooth=False,clamp=True):
        value = self.esc_count(c,smooth)/self.max_iter
        return max(0.,min(value,1.)) if clamp else value
    def esc_count(self,c,smooth=False):
        z=0
        for i in range(self.max_iter):
            z=z**2+c
            if abs(z)>self.radio_esc:
                if smooth:
                    return i +1-log(log(abs(z)))/log(2)
                return i
        return self.max_iter


In [32]:
@dataclass
class ViewPort:
    image: Image.Image
    center:complex
    width:float
    @property
    def height(self):
        return self.scale*self.image.height
    @property
    def offset(self):
        return self.center+complex(-self.width, self.height)/2
    @property
    def scale(self):
        return self.width/self.image.width
    
    def __iter__(self):
        for y in range(self.image.height):
            for x in range(self.image.width):
                yield Pixel(self,x,y)
    


In [34]:
@dataclass
class Pixel:
    viewport:ViewPort
    x:int
    y:int
    
    @property
    def color(self):
        return self.viewport.image.getpixel((self.x,self.y))
    @color.setter
    def color(self,value):
        self.viewport.image.putpixel((self.x,self.y),value)
    
    def __complex__(self):
        return ( complex(self.x,self.y)*self.viewport.scale+self.viewport.offset)
    

In [49]:
mandel=ConjMandelbrot(max_iter=256,radio_esc=1000)
image=Image.new(mode='L',size=(980,980))
for pixel in ViewPort(image,center=-0.7435+0.1314j,width=0.002):
    c=complex(pixel)
    instability=1-mandel.stability(c,smooth=True)
    pixel.color=int(instability*255)
    

In [50]:
image.show()

In [63]:
def paint(mandel,viewport,palette,smooth):
    for pixel in viewport:
        stability=mandel.stability(complex(pixel),smooth)
        index=int(min(stability*len(palette),len(palette) -1))
        pixel.color=palette[index % len(palette)]
                  

In [64]:
import matplotlib.cm

def denormalize(palette):
    return [tuple(int(channel * 255) for channel in color) for color in palette]

colormap=matplotlib.cm.get_cmap("twilight").colors
palette=denormalize(colormap)


<ipython-input-64-04f7e07551e7>:6: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap=matplotlib.cm.get_cmap("twilight").colors


In [67]:
image=Image.new(mode='RGB',size=(950,950))
viewport=ViewPort(image,center=-0.7435+0.1314j,width=0.002)
paint(mandel,viewport,palette,smooth=True)
image.show()